# Test weighted average

In [1]:
import sys
sys.path.insert(0,"/home/gridsan/hwpang/Software/RMG-Py/")

import json

import random
import time as time
import pandas as pd
import numpy as np

from rmgpy.molecule.group import Group
from rmgpy.data.base import Entry
from rmgpy.data.thermo import ThermoData

from thermo import ThermoGroups, average_thermo_data
from utils import make_mol
from parameters import Ts

# Get data

In [2]:
hbi_unc_df = pd.read_csv("../data/hbi_unc.csv")
hbi_unc_df

,radical_smiles,radical_resonance_smiles,closed_shell_smiles,closed_shell_H298 (kcal/mol),closed_shell_S298 (cal/mol/K),closed_shell_Cp300 (cal/mol/K),closed_shell_Cp400 (cal/mol/K),closed_shell_Cp500 (cal/mol/K),closed_shell_Cp600 (cal/mol/K),closed_shell_Cp800 (cal/mol/K),...,unc_HBI_H298 (kcal/mol),unc_HBI_S298 (cal/mol/K),unc_HBI_Sint298 (cal/mol/K),unc_HBI_Cp300 (cal/mol/K),unc_HBI_Cp400 (cal/mol/K),unc_HBI_Cp500 (cal/mol/K),unc_HBI_Cp600 (cal/mol/K),unc_HBI_Cp800 (cal/mol/K),unc_HBI_Cp1000 (cal/mol/K),unc_HBI_Cp1500 (cal/mol/K)
0,[O]C(=O)OC(O)(O)O,[O]C(=O)OC(O)(O)O,O=C(O)OC(O)(O)O,-272.617591,98.406448,31.562314,36.827897,41.095602,44.234417,48.821415,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1,[O]C(O)(O)OC(=O)O,[O]C(O)(O)OC(=O)O,O=C(O)OC(O)(O)O,-272.617591,98.406448,31.562314,36.827897,41.095602,44.234417,48.821415,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,O=C(O)O[C](O)O,O=C(O)O[C](O)O,O=C(O)OC(O)O,-231.363843,94.127419,27.214474,31.965583,36.032333,39.309560,44.059044,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,CC(=O)OC(OO)C(=O)C(O)O[O],CC(=O)OC(OO)C(=O)C(O)O[O],CC(=O)OC(OO)C(=O)C(O)OO,-234.219312,145.598548,52.706616,62.438776,71.442505,78.070363,87.443365,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
4,O=[C]OC(O)(O)O,O=[C]OC(O)(O)O,O=COC(O)(O)O,-216.499044,92.684651,26.935163,30.962696,34.637667,37.592447,42.375430,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2984,C1=C[CH]C=1,[C]1=CC=C1,C1=CC=C1,104.333500,57.818020,15.504000,19.920000,23.474000,26.261000,30.329000,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2985,C=C=C1C=[C]C1,C=C=C1C=[C]C1,C=C=C1C=CC1,91.366874,50.394958,22.416919,29.489266,35.000461,39.185145,45.162263,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2986,[CH2]C1=CC#CC1,[CH2]C1=CC#CC1,CC1=CC#CC1,131.227000,72.956831,23.409000,30.017000,34.619000,39.828000,46.529000,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2987,[CH2]C1=CC#CC1,C=C1[CH]C#CC1,C=C1CC#CC1,136.767000,72.435147,22.819000,29.017000,34.189000,38.448000,45.419000,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [26]:
trunc_hbi_unc_df = hbi_unc_df.head(2)

In [27]:
HBI_corrections = trunc_hbi_unc_df.apply(lambda x: ThermoData(
    H298=(x["HBI_H298 (kcal/mol)"], "kcal/mol", "+|-", x["unc_HBI_H298 (kcal/mol)"]),
    S298=(x["HBI_Sint298 (cal/mol/K)"], "cal/mol/K", "+|-", x["unc_HBI_Sint298 (cal/mol/K)"]),
    Cpdata=([x[f"HBI_Cp{T} (cal/mol/K)"] for T in Ts], "cal/mol/K", "+|-", [x[f"unc_HBI_Cp{T} (cal/mol/K)"] for T in Ts]),
    Tdata=(Ts, "K"),
    comment=f"Radical thermo from {x['radical_source']} and closed shell thermo from {x['closed_shell_thermo_source'].replace('Thermo library: ../data/', '')}"
), axis=1)
HBI_corrections = HBI_corrections.to_list()

In [28]:
average_thermo_data(HBI_corrections)

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([1.66672,1.15077,0.407867,0.152008,-0.253555,-0.635462,-2.54985],'cal/mol/K','+|-',[1.97382,1.15685,0.0142168,0.916509,2.22917,2.96339,3.26853]), H298=(49.1488,'kcal/mol','+|-',0.128319), S298=(-6.19891,'cal/mol/K','+|-',4.5517), comment="""Radical thermo from dong_pio_liang.py and closed shell thermo from GAV""")

In [29]:
average_thermo_data(HBI_corrections, weighted=True)

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([1.66672,1.15077,0.407867,0.152008,-0.253555,-0.635462,-2.54985],'cal/mol/K','+|-',[1.97382,1.15685,0.0142168,0.916509,2.22917,2.96339,3.26853]), H298=(49.1488,'kcal/mol','+|-',0.128319), S298=(-6.19891,'cal/mol/K','+|-',4.5517), comment="""Radical thermo from dong_pio_liang.py and closed shell thermo from GAV""")

In [30]:
HBI_corrections

[ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([2.36456,1.55978,0.412894,-0.172027,-1.04168,-1.68318,-3.70545],'cal/mol/K','+|-',[3,3,3,3,3,3,3]), H298=(49.1035,'kcal/mol','+|-',5.2), S298=(-4.58964,'cal/mol/K','+|-',3), comment="""Radical thermo from dong_pio_liang.py and closed shell thermo from GAV"""),
 ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([0.968866,0.741762,0.402841,0.476043,0.534574,0.412253,-1.39425],'cal/mol/K','+|-',[3,3,3,3,3,3,3]), H298=(49.1942,'kcal/mol','+|-',5.2), S298=(-7.80818,'cal/mol/K','+|-',3), comment="""Radical thermo from dong_pio_liang.py and closed shell thermo from GAV""")]